In [ ]:
train = read.csv(file = '../input/457-engineered/train_engineered (2).csv', header = TRUE)
test = read.csv(file = '../input/457-engineered/test_engineered (2).csv', header = TRUE)
train_sub = train[train$Sales > 0, ]#only use positive Sales in training data
testOpen = (test$Open == 1)
test_sub = test[testOpen, ] #only needs to do prediction for open days
colnames(train_sub)

In [ ]:
set.seed(123)
trainID = sample(1:nrow(train_sub), round(nrow(train_sub)*3/4))

library(gbm)
boost.data=gbm(Sales ~ ., 
                 data = train_sub[trainID,],
                 distribution="gaussian", 
                 n.tree=3000,
                 interaction.depth=3,
                 cv.folds = 5)
cv.num = gbm.perf(boost.data)

In [ ]:
library(MLmetrics)
yhat.boost.cv=predict(boost.data,newdata=train_sub[-trainID,],n.trees=cv.num)
RMSPE(y_pred = yhat.boost.cv, y_true = train_sub[-trainID,3])

In [ ]:
# Hypertruning Parameters

library(caret)
ctrl <- trainControl(method = "repeatedcv",
                     number = 10,
                     repeats = 5)

gbm.Grid = expand.grid(interaction.depth = c(2,3,4,5), 
                      n.trees = (1:5)*200, 
                      shrinkage = c(0.1, 0.05),
                      n.minobsinnode = 10) 


In [ ]:
gbm.cv.model <- train(Sales ~ ., data = train_sub[trainID,],
                     method = "gbm",
                     trControl = ctrl,
                     tuneGrid = gbm.Grid,
                     verbose = FALSE)
gbm.cv.model

In [ ]:
boost.pred = predict(gbm.cv.model, newdata =train_sub[-trainID,] )
RMSPE(y_pred = boost.pred, y_true = train_sub[-trainID,3])

# Customers

In [ ]:
set.seed(123)
trainID = sample(1:nrow(train_sub), round(nrow(train_sub)*3/4))

library(gbm)
boost.data=gbm(Customers ~ ., 
                 data = train_sub[trainID,-3],
                 distribution="gaussian", 
                 n.tree=1500,
                 interaction.depth=3,
                 cv.folds = 5)
cv.num = gbm.perf(boost.data)

In [ ]:
print(cv.num)

In [ ]:
library(MLmetrics)
yhat.boost.cv=predict(boost.data,newdata=train_sub[-trainID,-3],n.trees=cv.num)
RMSPE(y_pred = yhat.boost.cv, y_true = train_sub[-trainID,4])

0.066, 0.1326